## This file contains the final code for product recommendation

In [19]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Load Data (Replace paths with actual file paths)
sales_df = pd.read_csv('SALE DATA.csv', skipinitialspace=True)
items_df = pd.read_excel('CATEGORIES DATA.xlsx')

# Preprocess Product Names for Consistency
def clean_name(name):
    return name.strip().lower()

sales_df['Product Name'] = sales_df['Product Name'].apply(clean_name)
items_df['Item'] = items_df['Item'].apply(clean_name)

# Merge Sales Data with Categories
sales_df = sales_df.merge(
    items_df[['Item', 'Category']],
    left_on='Product Name',
    right_on='Item',
    how='left'
).drop(columns=['Item'])

# Calculate Average Price per Product
avg_price = sales_df.groupby('Product Name')['Price'].mean().reset_index()
avg_price.columns = ['Product Name', 'AvgPrice']

# Group Items by Cart (VchNo)
carts = sales_df.groupby('VchNo')['Product Name'].apply(list).reset_index()

# Build Co-occurrence Matrix
co_occurrence = defaultdict(lambda: defaultdict(int))
for cart in carts['Product Name']:
    for i in range(len(cart)):
        for j in range(i+1, len(cart)):
            product_a = cart[i]
            product_b = cart[j]
            co_occurrence[product_a][product_b] += 1
            co_occurrence[product_b][product_a] += 1

# Precompute Product Categories and Prices
product_category = sales_df.dropna(subset=['Category']).set_index('Product Name')['Category'].to_dict()
product_avg_price = avg_price.set_index('Product Name')['AvgPrice'].to_dict()

# Recommendation Logic
def recommend_products(input_product, top_n=5, price_range=0.2):
    input_product = clean_name(input_product)
    
    if input_product not in product_category or input_product not in co_occurrence:
        return []
    
    # Get input product details
    input_category = product_category[input_product]
    input_avg_price = product_avg_price.get(input_product, 0)
    price_low = input_avg_price * (1 - price_range)
    price_high = input_avg_price * (1 + price_range)
    
    # Step 1: Get co-occurring products
    candidates = co_occurrence[input_product]
    candidate_df = pd.DataFrame({
        'Product': list(candidates.keys()),
        'Frequency': list(candidates.values())
    })
    
    # Add category and price
    candidate_df['Category'] = candidate_df['Product'].map(product_category)
    candidate_df['AvgPrice'] = candidate_df['Product'].map(product_avg_price)
    candidate_df = candidate_df.dropna()
    
    # Step 2: Filter by category and price
    same_category = candidate_df[
        (candidate_df['Category'] == input_category) &
        (candidate_df['AvgPrice'].between(price_low, price_high))
    ].sort_values('Frequency', ascending=False)
    
    # Step 3: Fallback 1: Same category, any price
    if len(same_category) < top_n:
        same_category_fallback = candidate_df[
            (candidate_df['Category'] == input_category)
        ].sort_values('Frequency', ascending=False)
        same_category = pd.concat([same_category, same_category_fallback]).drop_duplicates()
    
    # Step 4: Fallback 2: Any category, similar price
    if len(same_category) < top_n:
        price_filtered = candidate_df[
            candidate_df['AvgPrice'].between(price_low, price_high)
        ].sort_values('Frequency', ascending=False)
        same_category = pd.concat([same_category, price_filtered]).drop_duplicates()
    
    # Step 5: Fallback 3: Top in category
    if len(same_category) < top_n:
        top_category = sales_df[sales_df['Category'] == input_category]
        top_products = top_category.groupby('Product Name').size().reset_index(name='Frequency')
        top_products = top_products[top_products['Product Name'] != input_product]
        same_category = pd.concat([same_category, top_products]).drop_duplicates()
    
    # Finalize and return top N
    recommendations = same_category.head(top_n)
    return recommendations[['Product', 'Category', 'AvgPrice', 'Frequency']].to_dict('records')

# Example Usage
input_product = "Veet  Hair Removal CrmNormal30GM(99*144)"
recommendations = recommend_products(input_product)
print(f"Recommendations for '{input_product}':")
for idx, rec in enumerate(recommendations, 1):
    print(f"{idx}. {rec['Product']} (Category: {rec['Category']}, Avg Price: ₹{rec['AvgPrice']:.2f}, Frequency: {rec['Frequency']})")

Recommendations for 'Veet  Hair Removal CrmNormal30GM(99*144)':
1. vivel aloe vera soap b4g1 (110*36) (Category: Bath and Body, Avg Price: ₹70.62, Frequency: 2)
2. odomos 100 gm cream vitamin-e(110*36) (Category: Skin Care, Avg Price: ₹59.86, Frequency: 2)
3. odonil  airfreshner 48gm po4(192*48) (Category: Household Supplies, Avg Price: ₹79.06, Frequency: 1)
4. gulcond plain 450gm+50gm(135*36) (Category: Juices, Avg Price: ₹77.10, Frequency: 1)
5. nan (Category: nan, Avg Price: ₹nan, Frequency: 16)


## Testing the above recommender with different test cases

In [16]:
input_product = "Dabur Red Paste 300Gm Offer(206*24)"
recommendations = recommend_products(input_product)
print(f"Recommendations for '{input_product}':")
for idx, rec in enumerate(recommendations, 1):
    print(f"{idx}. {rec['Product']} (Category: {rec['Category']}, Avg Price: ₹{rec['AvgPrice']:.2f}, Frequency: {rec['Frequency']})")


Recommendations for 'Dabur Red Paste 300Gm Offer(206*24)':
1. dabur red paste 37g offer (20*144) (Category: Oral Care, Avg Price: ₹12.99, Frequency: 5)
2. dabur red paste 100g  offer (70*72) (Category: Oral Care, Avg Price: ₹44.55, Frequency: 4)
3. dabur red paste 100g (70*72) (Category: Oral Care, Avg Price: ₹43.94, Frequency: 1)
4. vivel aloe vera soap b4g1 (110*36) (Category: Bath and Body, Avg Price: ₹70.62, Frequency: 21)
5. odonil  airfreshner 48gm po4(192*48) (Category: Household Supplies, Avg Price: ₹79.06, Frequency: 8)


In [18]:
input_product = "Lux White FlawlessGlow (PO4)41Gm(40*54)"
recommendations = recommend_products(input_product)
print(f"Recommendations for '{input_product}':")
for idx, rec in enumerate(recommendations, 1):
    print(f"{idx}. {rec['Product']} (Category: {rec['Category']}, Avg Price: ₹{rec['AvgPrice']:.2f}, Frequency: {rec['Frequency']})")

Recommendations for 'Lux White FlawlessGlow (PO4)41Gm(40*54)':
1. lux pink  radiant glow  (po4)41gm(40*54) (Category: Beauty and Cosmetics, Avg Price: ₹28.46, Frequency: 4)
2. livon serum 2 ml(4*480) (Category: Beauty and Cosmetics, Avg Price: ₹2.07, Frequency: 6)
3. lifebuoy strong red (po4) 44g(40*54 (Category: Bath and Body, Avg Price: ₹28.45, Frequency: 34)
4. surf excel bar 250gm(38*60) (Category: Laundry Supplies, Avg Price: ₹27.50, Frequency: 4)
5. parachute coconut oil 100ml ej( 44*180) (Category: Skin Care, Avg Price: ₹31.88, Frequency: 2)
